# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [160]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [161]:
# Use this cell for any import statements you add

import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [4]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: For the AZ Lyrics page, it has a noarchive tag that is in place to have specific documents and information that they don't want to be viewable to everybody. With this in place, the information allowed to be scraped is allowed by the page. There is no information that is being blocked and disabled for us to pull the lyrics and artist information that is needed for this assignment. If AZ Lyrics did not want their information to be scraped or used, there would be request errors showing that we are blocked from being able to pull the data. 


In [27]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # Add to ensure that request was successful
    print("If 200, request was successful: ", r.status_code)

    # Parse HTML Data
    soup = BeautifulSoup(r.content, "html.parser")
    # This will loop through artist page and pull all links starting with /lyrics/ and append it to list
    for links in soup.find_all("a", href = re.compile("/lyrics/")):
        lyrics_pages[artist].append(links.attrs["href"])
    

If 200, request was successful:  200
If 200, request was successful:  200


Let's make sure we have enough lyrics pages to scrape. 

In [28]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [29]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For robyn we have 104.
The full pull will take for this artist will take 0.29 hours.
For cher we have 318.
The full pull will take for this artist will take 0.88 hours.


In [30]:
print(lyrics_pages)

defaultdict(<class 'list'>, {'robyn': ['/lyrics/robyn/bumpyride.html', '/lyrics/robyn/inmyheart.html', '/lyrics/robyn/youvegotthatsomething.html', '/lyrics/robyn/doyouknowwhatittakes.html', '/lyrics/robyn/thelasttime.html', '/lyrics/robyn/justanothergirlfriend.html', '/lyrics/robyn/dontwantyouback.html', '/lyrics/robyn/doyoureallywantmeshowrespect.html', '/lyrics/robyn/how.html', '/lyrics/robyn/herewego.html', '/lyrics/robyn/wheredidourlovego.html', '/lyrics/robyn/robynishere.html', '/lyrics/robyn/iwish.html', '/lyrics/robyn/showmelove.html', '/lyrics/robyn/play.html', '/lyrics/robyn/myonlyreason.html', '/lyrics/robyn/underneaththeheart.html', '/lyrics/robyn/electric.html', '/lyrics/robyn/mytruth.html', '/lyrics/robyn/mainthing.html', '/lyrics/robyn/healthylove.html', '/lyrics/robyn/mondaymorning.html', '/lyrics/robyn/givingyouback.html', '/lyrics/robyn/88days.html', '/lyrics/robyn/longgone.html', '/lyrics/robyn/notontheinside.html', '/lyrics/robyn/universalwoman.html', '/lyrics/robyn/

## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [31]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [32]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [164]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

pages = 0

for artist, lyrics in lyrics_pages.items():

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    if os.path.exists("lyrics/" + artist + "/") == True:
        print("Folder Already Exists")
    else:
        os.mkdir("/lyrics/" + artist + "/")
        
    # 2. Iterate over the lyrics pages

    for page in lyrics:
        print("Web Scraping Currently: ", page)
        # Join URL_Stub to Lyrics Pages
        url = url_stub + page
        r = requests.get(url)
        time.sleep(5 + 10*random.random())

        # Iteration
        pages += 1

        # 3. Request the lyrics page. 
        soup = BeautifulSoup(r.text, "html.parser")
        
        
        # 4. Extract the title and lyrics from the page.
        # Extract Title
        title = soup.find_all("b")[1].text.strip('\"').replace(" ", "_")
    
        # Extract Lyrics
        lyrics_file = soup.find("div", attrs={"class": None, "id": None}).text
        lyrics_file = lyrics_file.replace("\n", " ").replace("\r", " ")

    
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url` to generate the filename. 
        file_name = "lyrics/" + artist + "/" + generate_filename_from_link(title)
        
        if artist == "robyn":
            output = str(title + "\n" + "\n" + lyrics_file)
            with open(f"{file_name}", "w+") as file:
                file.write(output)
        else:
            output = str(title + "\n" + "\n" + lyrics_file)
            with open(f"{file_name}", "w+") as file:
                file.write(output)
    

Folder Already Exists
Web Scraping Currently:  /lyrics/robyn/bumpyride.html
Web Scraping Currently:  /lyrics/robyn/inmyheart.html
Web Scraping Currently:  /lyrics/robyn/youvegotthatsomething.html
Web Scraping Currently:  /lyrics/robyn/doyouknowwhatittakes.html
Web Scraping Currently:  /lyrics/robyn/thelasttime.html
Web Scraping Currently:  /lyrics/robyn/justanothergirlfriend.html
Web Scraping Currently:  /lyrics/robyn/dontwantyouback.html
Web Scraping Currently:  /lyrics/robyn/doyoureallywantmeshowrespect.html
Web Scraping Currently:  /lyrics/robyn/how.html
Web Scraping Currently:  /lyrics/robyn/herewego.html
Web Scraping Currently:  /lyrics/robyn/wheredidourlovego.html
Web Scraping Currently:  /lyrics/robyn/robynishere.html
Web Scraping Currently:  /lyrics/robyn/iwish.html
Web Scraping Currently:  /lyrics/robyn/showmelove.html
Web Scraping Currently:  /lyrics/robyn/play.html
Web Scraping Currently:  /lyrics/robyn/myonlyreason.html
Web Scraping Currently:  /lyrics/robyn/underneaththehe

KeyboardInterrupt: 

I ended up stopping the for loop as the plan was to run all the songs, but Cher has significantly more than expected. Met the 20 requirement.

In [165]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.61 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [166]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [167]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For robyn we have 93 files.
For robyn we have roughly 26843 words, 2313 are unique.
For cher we have 92 files.
For cher we have roughly 16849 words, 1885 are unique.
